In [157]:
import numpy as np
import pandas as pd

# Game Deck

In [158]:
class GameDeck():
    """ Create the game deck """
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values, suits=4):
        self.num_decks = num_decks
        self.cards = values * num_decks * suits
        
    def __str__(self):
        return "{} cards left\n{}".format(len(self.cards), self.cards)
    
    def shuffle_cards(self):
        np.random.shuffle(self.cards)
        
    def deal_card(self, hand=None):
        if hand == None:
            return self.cards.pop(0)
        else:
            hand.append(self.cards.pop(0))

In [159]:
test = GameDeck()

In [160]:
print (test)
test.shuffle_cards()
print ()
print (test)
test.deal_card()
print()
print (test)

52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']

52 cards left
[3, 8, 8, 7, 'Q', 8, 'K', 6, 'J', 10, 2, 4, 8, 'J', 'J', 'K', 'K', 4, 5, 10, 'Q', 'K', 'A', 6, 6, 9, 4, 'J', 2, 5, 7, 'Q', 7, 10, 'A', 4, 3, 6, 2, 5, 'Q', 2, 7, 3, 'A', 5, 'A', 3, 10, 9, 9, 9]

51 cards left
[8, 8, 7, 'Q', 8, 'K', 6, 'J', 10, 2, 4, 8, 'J', 'J', 'K', 'K', 4, 5, 10, 'Q', 'K', 'A', 6, 6, 9, 4, 'J', 2, 5, 7, 'Q', 7, 10, 'A', 4, 3, 6, 2, 5, 'Q', 2, 7, 3, 'A', 5, 'A', 3, 10, 9, 9, 9]


# Points calculator

In [161]:
def calc_ace(hand, count_A):
    # For each Ace, see if adding 11 causes it to bust
    # If no, then take Ace as 11
    # Else, take Ace as 1
    working_hand = hand.copy()
    
    # Remove Aces
    working_hand = [i if type(i) == int else 10 for i in working_hand if i != "A"]
    for k in range(count_A):
        if sum(working_hand) + 11 <= 21:
            working_hand.append(11)
        else:
            working_hand.append(1)
    return working_hand

In [225]:
def calc_points(hand):
    try:
        hand = hand.split(",")
    except:
        pass
    
    working_hand = []
    
    for i in hand:
        try:
            working_hand.append(int(i))
        except:
            working_hand.append(i)
    
    if "A" in working_hand:
        
        # Count number of Aces
        count_A = working_hand.count("A")
        working_hand = calc_ace(working_hand, count_A)
        
    else:
        # Replace J, Q, and K with 10
        working_hand = [10 if i in ["J", "Q", "K"] else int(i) for i in working_hand]
    
    return sum(working_hand)

In [226]:
for i in [["A", "K"], [10,10], ["A", 5], ["A", "A", 8], [3,3]]:
    print (i, calc_points(i))

['A', 'K'] 21
[10, 10] 20
['A', 5] 16
['A', 'A', 8] 20
[3, 3] 6


# Soft 17

- Some casinos require the dealer to hit on a soft 17
- We need to know when it is a soft 17

In [258]:
def soft_17(hand):
    try:
        hand = hand.split(",")
    except:
        pass
    
    working_hand = []
    
    for i in hand:
        try:
            working_hand.append(int(i))
        except:
            working_hand.append(i)
    Ace_11 = False
    if calc_points(working_hand) == 17 and "A" in working_hand:
        count_A = working_hand.count("A")
        working_hand = calc_ace(working_hand, count_A)
        if 11 in working_hand[-count_A:]:
            return True
    return False

In [259]:
for i in [["K", 7], ["A", 3, 3], ["A", "A", 5], ["A", "A", "A", 4]]:
    print (i, soft_17(i))

['K', 7] False
['A', 3, 3] True
['A', 'A', 5] True
['A', 'A', 'A', 4] True


# Play game

In [260]:
def start_game(num_decks=1):
    game_deck = GameDeck(num_decks=num_decks)
    
    d_hand = []
    p_hand = []
    
    game_deck.shuffle_cards()
    
    for _ in range(2):
        game_deck.deal_card(p_hand)
        game_deck.deal_card(d_hand)
        
    return game_deck, p_hand, d_hand

In [261]:
start_game()

(<__main__.GameDeck at 0x110db89e8>, [7, 6], ['K', 2])

- strategies
    - 0: random
    - 1: recommended
    - 2: statistical analysis
    - 3: machine learning

In [262]:
def player_turn(game_deck, d_open, p_hand, strategy=0):
    new_hand = p_hand.copy()
    while calc_points(new_hand) <= 11:
        game_deck.deal_card(new_hand)
        
    if strategy == 0:
        while calc_points(new_hand) <= 18:
            if np.random.random() <= 0.5:
                game_deck.deal_card(new_hand)
            else:
                break
    elif strategy == 1:
        while calc_points(new_hand) <= 18:
            if d_open <= 6:
                game_deck.deal_card(new_hand)
            else:
                break
                
    return game_deck, p_hand, new_hand

In [263]:
test = start_game()
p_test = player_turn(test[0], test[1][0], test[2])
print (p_test)
print (calc_points(p_test[1]))
print (calc_points(p_test[2]))

(<__main__.GameDeck object at 0x110ddfbe0>, [7, 10], [7, 10])
17
17


In [329]:
def dealer_turn(game_deck, hand, soft=1):
    new_hand = hand.copy()
    if soft == 1:
        while calc_points(new_hand) < 17 or soft_17(new_hand):
            game_deck.deal_card(new_hand)
    else:
        while calc_points(new_hand) < 17:
            game_deck.deal_card(new_hand)
    return game_deck, hand, new_hand

In [330]:
test = start_game()
p_test = player_turn(test[0], test[1][0], test[2])
print (p_test)
print (calc_points(p_test[1]))
print (calc_points(p_test[2]))
d_test = dealer_turn(p_test[0], test[1])
print (d_test)
print (calc_points(d_test[1]))
print (calc_points(d_test[2]))

(<__main__.GameDeck object at 0x110de4da0>, [4, 'A'], [4, 'A'])
15
15
(<__main__.GameDeck object at 0x110de4da0>, [3, 10], [3, 10, 'A', 5])
13
19


In [331]:
def l_to_s(l):
    l = [str(i) for i in l]
    return ",".join(l)

In [423]:
def play_split(game_deck, p_hand, d_hand, d_open, num_decks, strategy=0, soft=1):
    to_split = np.random.random() <= 0.5
    if (strategy == 0 and to_split) or (strategy == 1 and p_hand[0] == 8):
        # Splits
        split = 1
        p_hand_1 = [p_hand[0]]
        p_hand_2 = [p_hand[1]]
        game_deck.deal_card(p_hand_1)
        game_deck.deal_card(p_hand_2)
        game_deck, p_hand_1, p_new_hand_1 = player_turn(game_deck, d_open, p_hand_1, strategy=strategy)
        game_deck, p_hand_2, p_new_hand_2 = player_turn(game_deck, d_open, p_hand_2, strategy=strategy)

        # If either is below 21, dealer gets to play
        if calc_points(p_new_hand_1) <= 21 or calc_points(p_new_hand_2) <= 21:
            game_deck, d_hand, d_new_hand = dealer_turn(game_deck, d_hand)
            # If hand 1 is above 21
            if calc_points(p_new_hand_1) > 21:
                # Dealer doesn't get to play for this hand
                d_new_hand_1 = d_hand.copy()
                out_1 = np.array([l_to_s(d_hand), l_to_s(d_new_hand_1), l_to_s(p_hand_1), l_to_s(p_new_hand_1), strategy, split, num_decks, soft])       
            else:
                out_1 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_1), l_to_s(p_new_hand_1), strategy, split, num_decks, soft])       
            # If hand 2 is above 21
            if calc_points(p_new_hand_2) > 21:
                # Dealer doesn't get to play for this hand
                d_new_hand_2 = d_hand.copy()
                out_2 = np.array([l_to_s(d_hand), l_to_s(d_new_hand_2), l_to_s(p_hand_2), l_to_s(p_new_hand_2), strategy, split, num_decks, soft])
            else:
                out_2 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_2), l_to_s(p_new_hand_2), strategy, split, num_decks, soft])       
        # They are both above 21, dealer doesn't play at all
        else:
            d_new_hand = d_hand.copy()
            out_2 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_2), l_to_s(p_new_hand_2), strategy, split, num_decks, soft])       
            out_1 = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_1), l_to_s(p_new_hand_1), strategy, split, num_decks, soft])       
            
        
        return np.array([out_1, out_2])
    else:
        return "No split"

In [424]:
def card_value(card):
    try:
        return int(card)
    except:
        if card == "A":
            return 1
        else:
            return 10

In [425]:
def play_game(num_decks=4, strategy=0, soft=1):
    game_deck, p_hand, d_hand = start_game(num_decks=num_decks)
    
    d_open = card_value(d_hand[0])
    split = 0
    if calc_points(p_hand) != 21 and calc_points(d_hand) != 21:
        if (p_hand[0] == p_hand[1]):
            out = play_split(game_deck, p_hand, d_hand, d_open, num_decks, strategy=strategy, soft=soft)
        else:
            out = "No split"
        if type(out) == str:
            game_deck, p_hand, p_new_hand = player_turn(game_deck, d_open, p_hand, strategy=strategy)
            if calc_points(p_new_hand) <= 21:
                game_deck, d_hand, d_new_hand = dealer_turn(game_deck, d_hand, soft=soft)
            else:
                d_new_hand = d_hand.copy()
            out = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand), l_to_s(p_new_hand), strategy, split, num_decks, soft])
    else:
        p_new_hand = p_hand.copy()
        d_new_hand = d_hand.copy()
        out = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand), l_to_s(p_new_hand), strategy, split, num_decks, soft])
    
    return out

In [426]:
play_game(strategy=1)

array(['2,10', '2,10', 'J,3', 'J,3,Q', '1', '0', '4', '1'], 
      dtype='<U5')

In [427]:
a = play_game()
while a.ndim < 2:
    a = play_game()
print (a)

[['8,10' '8,10' '8,Q' '8,Q' '0' '1' '4' '1']
 ['8,10' '8,10' '8,10' '8,10,4' '0' '1' '4' '1']]


In [428]:
a.ndim

2

# Generate cards

In [429]:
def gen_df(num_games=5000):
    data = []
    for _ in range(num_games):
        a = play_game(strategy=np.random.randint(2), soft=np.random.randint(2))
        if a.ndim == 2:
            data.extend(a)
        else:
            data.append(a)
            
    df = pd.DataFrame(data, columns=["d_hand", "d_final_hand", "p_hand", "p_final_hand", "strategy", "split", "num_decks", "soft_17"])
    return df

In [430]:
gen_df(num_games=1)

,d_hand,d_final_hand,p_hand,p_final_hand,strategy,split,num_decks,soft_17
0,"9,J","9,J","2,8","2,8,8,J",0,0,4,1


# Add columns

In [431]:
def add_columns(original):
    df = original.copy()
    df["strategy"] = df["strategy"].astype(int)
    df["split"] = df["split"].astype(int)
    df["num_decks"] = df["num_decks"].astype(int)
    df["soft_17"] = df["soft_17"].astype(int)
    
    df["d_open"] = df["d_hand"].apply(lambda x: card_value(x.split(",")[0]))
    df["d_initial"] = df["d_hand"].apply(calc_points)
    df["d_final"] = df["d_final_hand"].apply(calc_points)
    df["d_hit"] = df["d_final"]-df["d_initial"] > 0
    df["d_hit"] = df["d_hit"].astype(int)
    df["d_bust"] = df["d_final"] > 21
    df["d_bust"] = df["d_bust"].astype(int)
    
    df["p_initial"] = df["p_hand"].apply(calc_points)
    df["p_final"] = df["p_final_hand"].apply(calc_points)
    df["p_hit"] = df["p_final"]-df["p_initial"] > 0
    df["p_hit"] = df["p_hit"].astype(int)
    df["p_bust"] = df["p_final"] > 21
    df["p_bust"] = df["p_bust"].astype(int)
    
    df["p_win"] = (df["p_final"] > df["d_final"]) & (df["p_bust"] == 0) | (df["d_bust"] == 1)
    df["p_win"] = df["p_win"].astype(int)

    return df

# Generate CSVs

In [432]:
df = gen_df()

In [205]:
try:
    old_df = pd.read_csv("cards.csv")
    df = pd.concat([df, old_df])
except:
    pass
df.to_csv("cards.csv", index=False)

In [206]:
df = pd.read_csv("cards.csv")

In [207]:
df.shape

(5112, 8)

In [433]:
df2 = add_columns(df)

In [389]:
df2.to_csv("blackjack_data.csv")

# Test data

## Check hands and points

In [434]:
print (df2[["d_hand", "d_initial"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['8,3' 11]
 ['K,J' 20]
 ['8,K' 18]
 ['8,6' 14]
 ['A,3' 14]]


In [435]:
print (df2[["d_final_hand", "d_final"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['9,J' 19]
 ['9,10' 19]
 ['2,A' 13]
 ['8,K' 18]
 ['A,2,8' 21]]


In [436]:
print (df2[["p_hand", "p_initial"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['2,2' 4]
 ['7,K' 17]
 ['5,3' 8]
 ['7,9' 16]
 ['2,9' 11]]


In [437]:
print (df2[["p_final_hand", "p_final"]].as_matrix()[np.random.randint(len(df2), size=5)])

[['7,8' 15]
 ['5,9' 14]
 ['8,J' 18]
 ['2,7,K' 19]
 ['3,J' 13]]


## If p_bust = 1, p_win = 0

In [438]:
df2.p_win[df2["p_bust"] == 1].value_counts()

0    1057
Name: p_win, dtype: int64

In [439]:
df2[(df2["p_bust"] == 1) & df2.p_win == 1]

,d_hand,d_final_hand,p_hand,p_final_hand,strategy,split,num_decks,soft_17,d_open,d_initial,d_final,d_hit,d_bust,p_initial,p_final,p_hit,p_bust,p_win


## If d_bust = 1, p_win = 1

In [440]:
df2.p_win[df2["d_bust"] == 1].value_counts()

1    1039
Name: p_win, dtype: int64